In [ ]:
import os
from tensorflow import keras
import numpy as np
import math


from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Flatten, Dense, Dropout,BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, concatenate
from tensorflow.keras import optimizers, regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.initializers import he_normal
from tensorflow.keras.callbacks import LearningRateScheduler, TensorBoard, ModelCheckpoint

In [ ]:
num_classes        = 10
batch_size         = 64         # 64 or 32 or other
epochs             = 300
iterations         = 782       
USE_BN=True
LRN2D_NORM = True
DROPOUT=0.4
CONCAT_AXIS=3
WEIGHT_DECAY=1e-4
DATA_FORMAT='channels_last' # Theano:'channels_first' Tensorflow:'channels_last'

In [ ]:
log_filepath  = './inception_v1'

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
%matplotlib inline

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

plt.imshow(x_train[156])

In [ ]:
x_test.shape

In [ ]:
x_train.shape

In [ ]:
y_test.shape

In [ ]:
%matplotlib inline

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

In [ ]:
for i in range(1,11):
    index = np.where(y_train == i-1 )[0]
    for j in range(1,6):
        plt.figure(1, figsize=(15,15))
        plt.subplot( 10,5,(i-1)*5+j )
        plt.imshow(x_train[index[j]]  )
        
plt.show()
    

In [ ]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test  = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
#x_train = x_train/255

In [ ]:
def inception_module(x, nf1,nf2,nf3,nf4):
    
    branch1 = Conv2D( filters= nf1 , kernel_size=(1,1),strides=1,padding="same", activation="relu")(x)

    branch2 = Conv2D( filters= nf2[0] , kernel_size=(1,1),strides=1,padding="same", activation="relu")(x)
    branch2 = Conv2D( filters= nf2[1] , kernel_size=(3,3),strides=1,padding="same", activation="relu")(branch2)

    branch3 = Conv2D( filters= nf3[0] , kernel_size=(1,1),strides=1,padding="same", activation="relu")(x)
    branch3 = Conv2D( filters= nf3[1] , kernel_size=(5,5),strides=1,padding="same", activation="relu")(branch3)

    branch4 = MaxPooling2D(pool_size=(3,3),strides=1,padding="same")(x)
    branch4 = Conv2D(filters= nf4,kernel_size=(1,1),strides=1,padding="same", activation="relu" )(branch4)

    concatenation = concatenate([branch1,branch2,branch3,branch4])
    
    return concatenation

In [ ]:
def inception_module(x, nf1,nf2,nf3,nf4):
    
    branch1 = Conv2D( filters= nf1 , kernel_size=(1,1),strides=1,padding="same", activation="relu")(x)

    branch2 = Conv2D( filters= nf2[0] , kernel_size=(1,1),strides=1,padding="same", activation="relu")(x)
    branch2 = Conv2D( filters= nf2[1] , kernel_size=(3,3),strides=1,padding="same", activation="relu")(branch2)

    branch3 = Conv2D( filters= nf3[0] , kernel_size=(1,1),strides=1,padding="same", activation="relu")(x)
    branch3 = Conv2D( filters= nf3[1] , kernel_size=(5,5),strides=1,padding="same", activation="relu")(branch3)

    branch4 = MaxPooling2D(pool_size=(3,3),strides=1,padding="same")(x)
    branch4 = Conv2D(filters= nf4,kernel_size=(1,1),strides=1,padding="same", activation="relu" )(branch4)

    concatenation = concatenate([branch1,branch2,branch3,branch4])
    
    return concatenation

def create_model(input_image):

    x = Conv2D( filters= 64 , kernel_size=(7,7),strides=2,padding="same", activation="relu")(input_image)
    x = MaxPooling2D( pool_size=(3,3),strides=2,padding='same')(x)
    x = BatchNormalization()(x)

    x = Conv2D(filters=64,kernel_size=(1,1),strides=1,padding='same',activation='relu')(x)
    x = Conv2D(filters=192,kernel_size=(3,3),strides=1,padding='same',activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D( pool_size=(3,3),strides=2,padding='same')(x)

    x = inception_module(x,nf1 =64, nf2= (96,128), nf3 = (16,32), nf4=32   )
    x = inception_module(x,nf1 =128, nf2= (128,192), nf3 = (32,96), nf4=64   )
    x = MaxPooling2D( pool_size=(3,3),strides=2,padding='same')(x)

    x = inception_module(x,nf1 =192, nf2= (96,208), nf3 = (16,48), nf4=64   )
    x = inception_module(x,nf1 =160, nf2= (112,224), nf3 = (24,64), nf4=64   )
    x = inception_module(x,nf1 =128, nf2= (128,256), nf3 = (24,64), nf4=64   )
    x = inception_module(x,nf1 =112, nf2= (144,288), nf3 = (32,64), nf4=64   )
    x = inception_module(x,nf1 =256, nf2= (160,320), nf3 = (32,128), nf4=128   )
    x = MaxPooling2D(pool_size=(3,3),strides=2,padding='same')(x)

    x = inception_module(x,nf1 =256, nf2= (160,320), nf3 = (32,128), nf4=128   )
    x = inception_module(x,nf1 =384, nf2= (192,384), nf3 = (48,128), nf4=128   )
    x = AveragePooling2D(pool_size=(1,1),strides=1,padding="valid")(x)

    x=Flatten()(x)
    x=Dropout(DROPOUT)(x)

    x=Dense(units=num_classes,activation='linear')(x)
    x=Dense(units=num_classes,activation='softmax')(x)
    
    return x



In [ ]:
img_input=Input(shape=(32,32,3))
output = create_model(img_input)

model=Model(img_input,output)
model.summary()

In [ ]:
x_train = x_train/255
datagen = ImageDataGenerator(horizontal_flip=True,
                             width_shift_range=0.125,
                             height_shift_range=0.125,
                             fill_mode='constant',cval=0.)
#datagen.fit(x_train)

In [ ]:
x_train = x_train*255

In [ ]:
def scheduler(epoch):
    #if epoch < 150:
    #    return 0.1
    #if epoch < 225:
    #    return 0.01
    return 0.01

In [ ]:
# set optimizer
sgd = optimizers.SGD(lr=.1, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# set callback
tb_cb = TensorBoard(log_dir=log_filepath, histogram_freq=0)
change_lr = LearningRateScheduler(scheduler)
cbks = [change_lr,tb_cb]

In [ ]:
model.fit_generator(datagen.flow(x_train, y_train,batch_size=batch_size),
                    steps_per_epoch=iterations,
                    epochs=epochs,
                    callbacks=cbks,
                    validation_data=(x_test, y_test))